# Word Embedding 필요성
- Onehot Encoding 의 경우 Embedding Vector 의 열 개수가 약 700000 개로, Memory Error 및 학습 속도가 떨어지고 그 결과 또한 정확도가 매우 낮았다.
- 이러한 문제는 Onehot Encoding 의 결과가 Sparse Matrix (희소 행렬)을 만들기 때문으로, 위 문제를 해결하기 위해 Dense Matrix 를 사용하기로 했다.
    - Word Embedding 중에서도 가장 널리 쓰이는 Word2Vec 을 사용해 임베딩 벡터를 생성.
- [참고 자료](https://wikidocs.net/102705)

In [20]:
from sklearn.model_selection import train_test_split
import json
import pandas as pd

with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
plylst_data = pd.DataFrame(json_data)
plylst_data = plylst_data.drop(['plylst_title'], axis=1)
plylst_data = plylst_data.sample(frac=0.3)
plylst_data = plylst_data.reset_index()
plylst_data = plylst_data.sort_index()
plylst_data.head()
#plylst_data.shape

,index,tags,id,songs,like_cnt,updt_date
0,58673,"[기분전환, 스트레스]",90220,"[700090, 433453, 125822, 440239, 193571, 27358...",30,2017-03-17 12:46:05.000
1,87906,"[칸초네, ilvolo, 일볼로, 사랑]",47837,"[373821, 34647, 196415, 586450, 508896, 701572]",3,2016-11-11 01:40:02.000
2,13221,[뉴에이지],126227,"[543834, 502063, 416481, 91548, 298132, 697042...",1,2014-06-30 10:22:09.000
3,64119,"[센티멘탈, 추적추적, 비, Pop, 차분함]",39558,"[300174, 319815, 125425, 529790, 477732, 29491...",5,2020-04-10 17:16:01.000
4,114253,"[RnB, 감각적인, Pop, HipHop, 기분전환, Rock, 트렌디]",47871,"[492140, 692029, 110383, 185746, 570699, 61472...",24,2018-06-17 22:56:58.000


In [21]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
song_data = pd.DataFrame(json_data)
song_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


In [22]:
from collections import Counter

def extract_data(plylst_data, col_name):
    data_col = plylst_data[col_name]
    #train_tag = plylst_data['tags']
    datas = [data for data_list in data_col for data in data_list ]
    #tag_list = [tag for plist in train_tag for tag in plist]
    
    count_data = Counter(datas)
    #count_tag = Counter(tag_list)
    x = {}
    for key, value in count_data.items():
        if value>1:
            x[key]=value
    song_len = len(x)
    #for key, value in count_tag.items():
    #    if value>1:
    #        x[key]=value
    #tag_len = len(x) - song_len
    return list(x.keys()), song_len

song_extract_list, song_len = extract_data(plylst_data, 'tags')
print(song_extract_list)

['기분전환', '스트레스', '칸초네', '사랑', '뉴에이지', '센티멘탈', '비', 'Pop', '차분함', 'RnB', '감각적인', 'HipHop', 'Rock', '트렌디', '외힙', '국힙', '새벽', '추억', '밤', '회상', '셋리스트', '공연', '인디', '랩', '잔잔한', '드라이브', '알앤비', '재즈', '눈물', '이별노래', '작별', '눈물나는이별노래', '이별', '헤어짐', '눈물나는노래', '풋풋한', '하이틴', '설렘', '10대', '외출', '팝', '편안한', '테크노', '몽환', '앰비언트', '일렉', '추천', '소장', '노동요', '카페', '취향저격', '일할때', '최신', '매장음악', '신인', '휴식', '힐링', '여행', '산책', '비오는날', '포크', '슬픔', '조용한', '우울', '느낌있는', '흔하지않은', '유니크', '유니크한', '세련된', '가을', '락', 'OST', '애니메이션', '소울', '까페', '힙합', '감성', '스타일리쉬', '날씨', '매장', '피쳐링', '멜로디', '인디밴드', '보컬재즈', '겨울', '따뜻한밤', '싸이월드배경음악', '좋은_노래', '싸이월드BGM', '라운지', '여름', '패션', '분위기깡패', '감성곡', '백예린', '분위기', '신곡', '최신곡', '크루셜스타', '콘서트', '멜론티켓', '한올', '발라드', '추억의가수', '그리움', '국내', '빌보드Hot50', '빌보드', '피아노연주', '태교', '클래식', '임산부', '태교음악', '밴드사운드', '밴드', '설', '정세운', '여름음악', '방탄소년단', '인디음악', '수란', 'nct', '여름송', '크러쉬', '가을노래', '11월', '낙엽', '설레는', '수고했어', '국내힙합', '연말파티', '파티', '네덜란드', '영국', '덴마크', '아일랜드', '스코틀랜드', '감성힙합', '알엔비', '일렉트로니카',

# Word2Vec Parameter

- vector_size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram.

In [23]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=plylst_data['tags'], vector_size=100, window=5, min_count=3, workers=4, sg=0)
res = model.wv.most_similar("록")

In [24]:
print(res)

[('Rock', 0.9920139908790588), ('메탈', 0.9899711608886719), ('Metal', 0.9893632531166077), ('록메탈', 0.9885344505310059), ('스쿨오브록', 0.9884926080703735), ('legend', 0.9812719225883484), ('락', 0.9791022539138794), ('영국', 0.9772225022315979), ('메들리', 0.9749714136123657), ('오피셜차트', 0.9747001528739929)]


In [25]:
import numpy as np

print(len(plylst_data))
plylst_data['tags'].replace('', np.nan, inplace=True)
plylst_data = plylst_data[plylst_data['tags'].notna()]
print(len(plylst_data))

34521
34521


# 발견한 문제

- 초기 데이터셋의 Index 와 Embedding Vector 값을 저장한 Index 의 차이가 존재해 결과에 영향을 미치는 상태
- Embedding Vector 에서 None 이 발생하는 이유를 찾아야 하는 것이 우선적으로 해결해야할 과제가 될 듯함.

In [26]:
from gensim.models import KeyedVectors

def get_vector(tag_list):
    embedding_list = []
    for tags in tag_list:
        avg_vec = None
        cnt = 0
        for tag in tags:
            if tag in model.wv:
                cnt += 1
                if avg_vec is None:
                    avg_vec = model.wv.get_vector(tag)
                else:
                    avg_vec = avg_vec + model.wv.get_vector(tag)
        
        if avg_vec is not None:
            avg_vec = avg_vec / cnt
            embedding_list.append(avg_vec)
        else:
            print("None")
    
    return embedding_list

embedding_list = get_vector(plylst_data['tags'])
print(len(embedding_list))
print(len(plylst_data))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [27]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarities = cosine_similarity(embedding_list, embedding_list)
print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)

코사인 유사도 매트릭스의 크기 : (33942, 33942)


In [29]:
def recommend(idx):
    sim_score = list(enumerate(cosine_similarities[idx]))
    sim_score = sorted(sim_score, key = lambda x: x[1], reverse = True)
    sim_score = sim_score[1:6]
    plist_idx = [i[0] for i in sim_score]
    print("Playlist {} 과 가장 유사한 플레이리스트 : {} {} {} {} {}".format(idx, plist_idx[0], plist_idx[1], plist_idx[2], plist_idx[3], plist_idx[4]))

recommend(2)
recommend(3)
recommend(4)

Playlist 2 과 가장 유사한 플레이리스트 : 222 235 277 305 344
Playlist 3 과 가장 유사한 플레이리스트 : 21017 33193 4261 2855 5660
Playlist 4 과 가장 유사한 플레이리스트 : 19508 31258 196 27731 931
